In [1]:
import numpy as np
import pickle
import scipy as sp
import scipy.sparse as spp

def load_sparse_matrix(filename):
    t = 0
    f = open(filename, 'r')
    f.readline()

    user_list = []
    item_list = []
    data_list = []
    count  = 0
    for line in f:
        word = line.split(',')
       
        rating = float(word[2])

        t += 1
        if t % 50000 == 0:
            print('.', end = '')
            if t % 2000000 == 0:
                print('%d m lines' %(t // 1000000))
            #data_list.append(float(rating))
        if rating > 4.0:
            data_list.append(1)
        elif rating < 2.0:
            data_list.append(-1)
            count +=1
        else: 
            continue    
        
        user_list.append(int(word[0]))
        item_list.append(int(word[1]))
        
    print(count)
    users = list(set(user_list))
    items = list(set(item_list))
    user2id = dict(zip(users, range(len(users))))
    item2id = dict(zip(items, range(len(items))))

    user_id_list = [user2id[u] for u in user_list]
    item_id_list = [item2id[i] for i in item_list]

    result = spp.csr_matrix((data_list, (user_id_list, item_id_list)))
    return result

def extract_users(num_users, sparse_matrix):
	item_count = sparse_matrix.getnnz(axis = 1)
	user_sort = sorted(enumerate(item_count), key = lambda x: x[1], reverse = True)[:num_users]
	user_indices = [x[0] for x in user_sort]

	return spp.vstack([sparse_matrix.getrow(i) for i in user_indices])

def extract_items(num_items, sparse_matrix):
	user_count = sparse_matrix.getnnz(axis = 0)
	item_sort = sorted(enumerate(user_count), key = lambda x: x[1], reverse = True)[:num_items]
	item_indics = [x[0] for x in item_sort]

	return spp.hstack([sparse_matrix.getcol(j) for j in item_indics])

def get_reduced_matrix(num_users, num_items, filename):
	data = load_sparse_matrix(filename)
	data1 = extract_users(3 * num_users, data)
	data2 = extract_items(num_items, data1)
	data3 = extract_users(num_users, data2)
	return data3.toarray()










In [2]:
m = get_reduced_matrix(num_users = 2000, num_items = 2000, filename = './ratings.csv')
print(m.shape)


........................................2 m lines
........................................4 m lines
........................................6 m lines
........................................8 m lines
........................................10 m lines
........................................12 m lines
........................................14 m lines
........................................16 m lines
........................................18 m lines
........................................20 m lines
1199109
(20000, 20000)


In [3]:
from sklearn.preprocessing import normalize

def ExtractFeatures(num, d, m):
    X = m

    A1 = X[:num, :]
    u, s, vt = np.linalg.svd(A1)

    u = u[:, :d-1]
    u = normalize(u, axis = 1, norm = 'l2')
    # print(np.linalg.norm(u[0,:]))

    U = np.concatenate((u, np.ones((num, 1))), axis = 1) / np.sqrt(2)
    return U

U = ExtractFeatures(2000, d=10, m = m)
np.save("../movielens/movie_2000users_2000items_features_10.npy", U)


c = m.T
I = ExtractFeatures(2000, d=10, m = c)
np.save("../movielens/movie_2000items_2000users_features_10.npy", I)

a = 0
b = 0
new_m = []
for i in range(len(m)):
    for j in range(len(m[0])):
        if m[i][j] > 0:
            new_m.append((i,j,1))
            a +=1
            
        if m[i][j] < 0:
            new_m.append((i,j,-1))
            b+=1
print(a, b)

np.save("../movielens/movie_2000users_2000items_entry_10.npy", np.array(new_m))



2243703 729924
